In [30]:
import os
import re
import json
import requests
import wolframalpha
from groq import Groq

In [52]:
os.environ["GROQ_API_KEY"] = "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"
groq_api_key=os.environ.get("GROQ_API_KEY")
model="llama-3.1-70b-versatile"

In [20]:
client = Groq(
    api_key=groq_api_key,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of language models",
        }
    ],
    model=model,
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in chat_completion:
    print(chunk.choices[0].delta.content or "", end="")

### Use model to create queries for Wolfram API

In [56]:
wolfram_app_id = "Y89ETU-L7Y6WK73H8"
os.environ["WOLFRAM_ALPHA_APPID"] = wolfram_app_id
client = wolframalpha.Client(wolfram_app_id)

In [31]:
def wolfram_api_call(client, solve):
    response = client.query(solve)
    full_response = response.json()
    pods = full_response["queryresult"]["pods"][1]["subpods"]
    return pods

In [8]:
from langchain.prompts import PromptTemplate
prompt = '''Use Wolfram alpha api calls to solve this: {question}'''

math_assitant_prompt = PromptTemplate(input_variables=["question"],
                                      template=prompt)

In [14]:
def get_wolfram_response(verbose_query, app_id):
    # Wolfram api fails with verbose query. RegEx to obtain query enclosed in quotes.
    queries = re.findall(r'"([^"]*)"', verbose_query)
    query = queries[0]
    params = {
        "input": query,
        "appid": app_id,
        "format": "plaintext",
        "output": "json",
    }
    response = requests.get("https://api.wolframalpha.com/v2/query", params=params)
    full_response = response.json()
    pods = full_response["queryresult"]["pods"][1]["subpods"]
    return pods

In [75]:
params = {
    "input": "Solve integ 4x^2 dx",
    "appid": wolfram_app_id,
    "format": "plaintext",
    "output": "json",
}
response = requests.get("https://api.wolframalpha.com/v2/query", params=params)

In [76]:
response.json()["queryresult"]["pods"]

[{'title': 'Indefinite integral',
  'scanner': 'Integral',
  'id': 'IndefiniteIntegral',
  'position': 100,
  'error': False,
  'numsubpods': 1,
  'primary': True,
  'subpods': [{'title': '',
    'plaintext': 'integral4 x^2 dx = (4 x^3)/3 + constant'}],
  'expressiontypes': {'name': 'Default'},
  'states': [{'name': 'Step-by-step solution',
    'input': 'IndefiniteIntegral__Step-by-step solution',
    'stepbystep': True,
    'buttonstyle': 'StepByStepSolution'}]},
 {'title': 'Plot of the integral',
  'scanner': 'Integral',
  'id': 'Plot',
  'position': 200,
  'error': False,
  'numsubpods': 1,
  'subpods': [{'title': '', 'plaintext': ''}],
  'expressiontypes': {'name': 'Default'}}]

In [51]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [53]:
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model=model)
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | llm
chain.invoke({"text": "Explain the importance of low latency LLMs."})

AIMessage(content='Low-latency Large Language Models (LLMs) are crucial for various applications, particularly those requiring real-time or near-real-time interactions. The importance of low-latency LLMs can be understood from the following aspects:\n\n1. **Improved User Experience**: Low-latency LLMs enable faster response times, making interactions feel more natural and engaging. This is particularly important for applications like chatbots, virtual assistants, and language translation, where users expect quick and accurate responses.\n\n2. **Real-time Decision Making**: In applications like autonomous vehicles, medical diagnosis, or financial trading, low-latency LLMs can provide critical information in real-time, enabling timely decision-making and potentially saving lives or preventing financial losses.\n\n3. **Enhanced Productivity**: Low-latency LLMs can accelerate tasks like language translation, text summarization, and content generation, allowing professionals to work more ef

In [59]:
from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper
wolfram = WolframAlphaAPIWrapper()
# does not work in jupyter because jupyter is asyncio loop
# wolfram.run("What is 2x+5 = -3x + 7?")

In [93]:
from langchain.tools import tool

@tool
def wolfram_call(query: str) -> str:
    """Calls the wolfram alpha api on query.

    Args:
        query (str): The query to use as a parameter in the function call. For example, "Solve 3x=5"

    Returns:
        str: Returns the result of the api call on the query.
    """
    params = {
        "input": query,
        "appid": wolfram_app_id,
        "format": "plaintext",
        "output": "json",
    }
    response = requests.get("https://api.wolframalpha.com/v2/query", params=params)
    full_response = response.json()
    pods = [x["subpods"] for x in full_response["queryresult"]["pods"]]
    return str(pods)

In [63]:
print(wolfram_call.name)
print(wolfram_call.description)
print(wolfram_call.args)

wolfram_call
Calls the wolfram alpha api on query.

    Args:
        query (str): The query to use as a parameter in the function call. For example, "Solve 3x=5"

    Returns:
        str: Returns the result of the api call on the query.
{'query': {'title': 'Query', 'type': 'string'}}


In [94]:
wolfram_call.run("What is the integral of 4x^2?")

"[[{'title': '', 'plaintext': 'integral4 x^2 dx = (4 x^3)/3 + constant'}], [{'title': '', 'plaintext': ''}]]"

In [106]:
from langchain.chains import create_structured_output_runnable
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a mathematical assistant. Please solve the questions in a step-by-step manner.
         Use the wolfram tool wherever it is possible to create a valid query to solve any algebra required in the steps of the problem."""),
        MessagesPlaceholder("chat_history", optional=True),
        ('human', '{input}'),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)


In [103]:

# structured_llm = create_structured_output_runnable(
#     wolfram_call,
#     llm,
#     prompt,
#     mode="openai-tools",
#     enforce_function_usage=True,
#     return_single=True
# )

# structured_llm.invoke({"input": "What is 3x^2 + 17x + 255 = 0?"})

In [107]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

toolkit = [wolfram_call]

agent = create_openai_tools_agent(llm, toolkit, prompt)

agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True)

In [108]:
result = agent_executor.invoke({"input": "Solve x^2 + 2x + 5 = 0"})



> Entering new AgentExecutor chain...

Invoking: `wolfram_call` with `{'query': 'Solve x^2 + 2x + 5 = 0'}`


[[{'title': '', 'plaintext': 'solve x^2 + 2 x + 5 = 0'}], [{'title': '', 'plaintext': 'x = -1 - 2 i'}, {'title': '', 'plaintext': 'x = -1 + 2 i'}], [{'title': '', 'plaintext': ''}], [{'title': '', 'plaintext': '-2'}], [{'title': '', 'plaintext': '5'}]]The solutions to the equation x^2 + 2x + 5 = 0 are x = -1 - 2i and x = -1 + 2i.

> Finished chain.


In [110]:
result2 = agent_executor.invoke({"input": "A drawer contains red socks and black socks. When two socks are drawn at random, the probability that both are red is 1/2. How small can the number of socks in the drawer be? "})



> Entering new AgentExecutor chain...

Invoking: `wolfram_call` with `{'query': '(r/n) * ((r-1)/(n-1)) = 1/2, solve for n and r, n and r are positive integers'}`


[[{'title': '', 'plaintext': 'solve r/n×(r - 1)/(n - 1) = 1/2 for n, r, n, r'}], [{'title': '', 'plaintext': 'n = 1/4 (-(1 + sqrt(2)) (3 - 2 sqrt(2))^(c_1) + (sqrt(2) - 1) (3 + 2 sqrt(2))^(c_1) + 2) and r = 1/8 ((2 + sqrt(2)) (3 - 2 sqrt(2))^(c_1) - (sqrt(2) - 2) (3 + 2 sqrt(2))^(c_1) + 4) and c_1>=2 and c_1 element Z'}], [{'title': '', 'plaintext': 'n = 4 and r = 3'}, {'title': '', 'plaintext': 'n = 21 and r = 15'}, {'title': '', 'plaintext': 'n = 120 and r = 85'}, {'title': '', 'plaintext': 'n = 697 and r = 493'}, {'title': '', 'plaintext': 'n = 4060 and r = 2871'}, {'title': '', 'plaintext': 'n = 23661 and r = 16731'}, {'title': '', 'plaintext': 'n = 137904 and r = 97513'}, {'title': '', 'plaintext': 'n = 803761 and r = 568345'}, {'title': '', 'plaintext': 'n = 4684660 and r = 3312555'}]]The smallest number of socks in 

In [111]:
result3 = agent_executor.invoke({"input": "Determine if the series of 1/(n^pi) converges or diverges. Explain each step."})



> Entering new AgentExecutor chain...

Invoking: `wolfram_call` with `{'query': 'pi > 1'}`


[[{'title': '', 'plaintext': 'π>1'}], [{'title': '', 'primary': True, 'plaintext': 'True'}]]
Invoking: `wolfram_call` with `{'query': 'sum of 1/(n^pi) from n=1 to infinity'}`


[[{'title': '', 'plaintext': 'sum_(n=1)^∞ 1/n^π = ζ(π)≈1.1762'}], [{'title': '', 'plaintext': 'sum_(n=1)^∞ 1/n^π converges'}], [{'title': '', 'plaintext': 'sum_(n=1)^k 1/n^π = H_k^(π)'}], [{'title': '', 'plaintext': ''}], [{'title': '', 'plaintext': 'ζ(π) = sum_(k=1)^∞ k^(-π)'}, {'title': '', 'plaintext': 'ζ(π) = -(2^π sum_(k=1)^∞ (-1)^k k^(-π))/(-2 + 2^π)'}, {'title': '', 'plaintext': 'ζ(π) = (2^π sum_(k=0)^∞ (1 + 2 k)^(-π))/(-1 + 2^π)'}, {'title': '', 'plaintext': 'ζ(π) = e^( sum_(k=1)^∞ P(k π)/k)'}]]The series 1/(n^pi) converges. 

Step 1: We first need to determine if the series is a p-series. A p-series is a series of the form 1/n^p. In this case, p = pi. 

Step 2: We need to determine if p is greater than 1. If p